In [ ]:
!wget https://github.com/ChanCheeKean/datasets/blob/main/h5/deep_learning_h5.zip?raw=true
!unzip -qq deep_learning_h5.zip?raw=true
!pip install datasets transformers

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, 
                                     Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, AveragePooling1D)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from lib.resnets_utils import *
# import public_tests

# 1) Resnett Overview

Residual Networks (ResNets), a very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow you to train much deeper networks than were previously feasible.

In recent years, neural networks have become much deeper, with state-of-the-art networks evolving from having just a few layers (e.g., AlexNet) to over a hundred layers.

* The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the shallower layers, closer to the input) to very complex features (at the deeper layers, closer to the output). 

* However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent prohibitively slow.

* More specifically, during gradient descent, as you backpropagate from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode," from gaining very large values). 

* During training, you might therefore see the magnitude (or norm) of the gradient for the shallower layers decrease to zero very rapidly as training proceeds, as shown below: 

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/vanishing_grad_kiank.png?raw=true" style="width:450px;height:220px;">
<caption><center> <u><b>Figure 1</b> </u> : <b>Vanishing gradient</b> <br> The speed of learning decreases very rapidly for the shallower layers as the network trains </center></caption>


In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:  

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/skip_connection_kiank.png?raw=true" style="width:650px;height:200px;">
<caption><center> <u> <b>Figure 2</b> </u>: A ResNet block showing a skip-connection <br> </center></caption>

The image on the left shows the "main path" through the network. The image on the right adds a shortcut to the main path. By stacking these ResNet blocks on top of each other, you can form a very deep network. 

The lecture mentioned that having ResNet blocks with the shortcut also makes it very easy for one of the blocks to learn an identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance.  
    
On that note, there is also some evidence that the ease of learning an identity function accounts for ResNets' remarkable performance even more than skip connections help with vanishing gradients.

# 2) The Identity Block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps:

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/idblock2_kiank.png?raw=true" style="width:650px;height:150px;">
<caption><center> <u><b>Figure 3</b> </u>: <b>Identity block.</b> Skip connection "skips over" 2 layers. </center></caption>

The upper path is the "shortcut path." The lower path is the "main path." In this diagram, notice the CONV2D and ReLU steps in each layer. To speed up training, a BatchNorm step has been added.

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/idblock3_kiank.png?raw=true" style="width:650px;height:150px;">
    <caption><center> <u><b>Figure 4</b> </u> : <b>Identity block.</b> Skip connection "skips over" 3 layers.</center></caption>

In [ ]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    cache = []
    # First component of main path
    X = Conv2D(filters=F1, kernel_size=1, strides = (1, 1), padding='valid', kernel_initializer=initializer())(X) # (m, w, h, filters)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=f, strides = (1,1), padding='same', kernel_initializer=initializer())(X) # (m, w, h, filters)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    # Third component of main path
    X = X = Conv2D(filters=F3, kernel_size=1, strides = (1,1), padding='valid', kernel_initializer=initializer())(X) # (m, w, h, filters)
    X = BatchNormalization(axis=3)(X, training=training)

    # Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [ ]:
np.random.seed(1)
X1 = np.ones((1, 28, 28, 3)) * -1
X2 = np.ones((1, 28, 28, 3)) * 1
X3 = np.ones((1, 28, 28, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[8, 8, 3], training=False)
A3np = A3.numpy()

# 3) The Convolutional Block

The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path: 

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/convblock_kiank.png?raw=true" style="width:650px;height:150px;">
<caption><center> <u><b>Figure 4</b> </u>: <b>Convolutional block</b> </center></caption>

* The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. (This plays a similar role as the matrix $W_s$ discussed in lecture.) 
* For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2. 
* The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step. 
* As for the previous exercise, the additional `initializer` argument is required for grading purposes, and it has been set by default to [glorot_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform)

In [ ]:
def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X # (m, h, w, 3)
    #################
    ### MAIN PATH ###
    #################

    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer())(X) # (m, h/2, w/2, f1)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # 2nd Components
    X = Conv2D(filters=F2, kernel_size=f, strides = (1, 1), padding='same', kernel_initializer = initializer())(X) # (m, h/2, w/2, f2)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    # 3rd Components
    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer = initializer())(X) # (m, h/2, w/2, f3)
    X = BatchNormalization(axis=3)(X, training=training)

    ### SHORTCUT PATH ###
    X_shortcut = Conv2D(filters=F3, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer = initializer())(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training) # (m, h/2, w/2,f3)
    
    # Add shortcut value to main path
    X = Add()([X, X_shortcut]) # (m, h/2, w/2, f3)
    X = Activation('relu')(X)
    return X

In [ ]:
X1 = np.ones((1, 28, 28, 3)) * -1
X2 = np.ones((1, 28, 28, 3)) * 1
X3 = np.ones((1, 28, 28, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)
A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

# 4) 50 Layers Resnet Model Building

<img src="https://github.com/sebastianbirk/coursera-deep-learning-specialization/blob/master/04_convolutional_neural_networks/04_resnets_with_keras/images/resnet_kiank.png?raw=true" style="width:850px;height:150px;">
<caption><center> <u> <b>Figure 5</b> </u>: <b>ResNet-50 model</b> </center></caption>

In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape) # (m, 64, 64, 3)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) # (m, 70, 70, 3) 
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform())(X) # (m, 32, 32, 64) 
    X = BatchNormalization(axis=3)(X) # (m, 32, 32, 64) 
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) # (m, 15, 15, 64) 

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1) # (m, 15, 15, 256) 
    X = identity_block(X, 3, [64, 64, 256]) # (m, 15, 15, 256) 
    X = identity_block(X, 3, [64, 64, 256]) # (m, 15, 15, 256) 
    
    # stage 3
    X = convolutional_block(X, f = 3, filters = [128,128,512], s=2) # (m, 8, 8, 512) 
    X = identity_block(X, 3, [128,128,512]) # (m, 8, 8, 512) 
    X = identity_block(X, 3, [128,128,512]) # (m, 8, 8, 512) 
    X = identity_block(X, 3, [128,128,512]) # (m, 8, 8, 512) 
    
    # stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s=2)  # (m, 4, 4, 1024) 
    X = identity_block(X, 3, [256, 256, 1024]) # (m, 4, 4, 1024) 
    X = identity_block(X, 3, [256, 256, 1024]) # (m, 4, 4, 1024) 
    X = identity_block(X, 3, [256, 256, 1024]) # (m, 4, 4, 1024) 
    X = identity_block(X, 3, [256, 256, 1024]) # (m, 4, 4, 1024) 
    X = identity_block(X, 3, [256, 256, 1024]) # (m, 4, 4, 1024) 
    
    # stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s=2) # (m, 2, 2, 2048) 
    X = identity_block(X, 3, [512, 512, 2048]) # (m, 2, 2, 2048) 
    X = identity_block(X, 3, [512, 512, 2048]) # (m, 2, 2, 2048) 
    
    # output layer
    X = AveragePooling2D()(X) # (m , 1, 1, 2048)
    X = Flatten()(X) # (m, 2048)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform())(X) # (m, classes)    
    model = Model(inputs = X_input, outputs = X)
    return model

# 5) Model Training

## 5.1. Training

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
# print(model.summary())

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [ ]:
model.fit(X_train, Y_train, epochs=2, batch_size=32)

Epoch 1/2
34/34 [==============================] - 86s 3s/step - loss: 1.8855 - accuracy: 0.5130
Epoch 2/2
34/34 [==============================] - 84s 2s/step - loss: 0.9937 - accuracy: 0.7093


## 5.2 Evaluation

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 304ms/step - loss: 1.1953 - accuracy: 0.5583
Loss = 1.1953405141830444
Test Accuracy = 0.5583333373069763


# 6) Load Pre-Trained Model

In [ ]:
pre_trained_model = tf.keras.models.load_model('resnet50.h5')

In [ ]:
preds = pre_trained_model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 252ms/step - loss: 0.1596 - accuracy: 0.9500
Loss = 0.15958662331104279
Test Accuracy = 0.949999988079071


<font color = 'blue'>

**What you should remember**:

- Very deep "plain" networks don't work in practice because vanishing gradients make them hard to train.  
- Skip connections help address the Vanishing Gradient problem. They also make it easy for a ResNet block to learn an identity function. 
- There are two main types of blocks: The **identity block** and the **convolutional block**. 
- Very deep Residual Networks are built by stacking these blocks together.

# 7) Vision Transformers
https://medium.com/@lucrece.shin/ch-9-vision-transformer-part-i-introduction-and-fine-tuning-in-pytorch-14674920c2ea

In [ ]:
import torch
from datasets import load_dataset
import random
from PIL import ImageDraw, ImageFont, Image
from transformers import ViTForImageClassification
from transformers import ViTFeatureExtractor
from transformers import TrainingArguments

def show_examples(ds, seed: int = 1234, examples_per_class: int = 3, size=(350, 350)):
    '''To print image in grid'''
    w, h = size
    labels = ds['train'].features['labels'].names
    grid = Image.new('RGB', size=(examples_per_class * w, len(labels) * h))
    draw = ImageDraw.Draw(grid)
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf", 24)

    for label_id, label in enumerate(labels):

        # Filter the dataset by a single label, shuffle it, and grab a few samples
        ds_slice = ds['train'].filter(lambda ex: ex['labels'] == label_id).shuffle(seed).select(range(examples_per_class))

        # Plot this label's examples along a row
        for i, example in enumerate(ds_slice):
            image = example['image']
            idx = examples_per_class * label_id + i
            box = (idx % examples_per_class * w, idx // examples_per_class * h)
            grid.paste(image.resize(size), box=box)
            draw.text(box, label, (255, 255, 255), font=font)

    return grid

ds = load_dataset('beans')
print("Data Summary: ", ds)
print("Sample Data", ds['train'][100])
print("Data Label Name", ds['train'][100])

show_examples(ds, seed=random.randint(0, 1337), examples_per_class=3)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# load feature extractor
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
print('\nFeature Extractor', feature_extractor)

# process an image
print('\nFeatures', feature_extractor(ds['train'][100]['image'], return_tensors='pt'))

# load model
labels = ds['train'].features['labels'].names
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

# modify the last layer for finetuning
# model.classifier = torch.nn.Linear(768, len(labels))
input = feature_extractor(ds['train'][800]['image'], return_tensors='pt')
output = model.forward(**input)
print('\nModel Output', output.logits)


Feature Extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}


Features {'pixel_values': tensor([[[[ 0.1294,  0.2314,  0.4980,  ..., -0.3020, -0.1843, -0.0824],
          [ 0.4275,  0.4431,  0.5216,  ..., -0.2941, -0.0510, -0.0039],
          [ 0.4275,  0.3255,  0.2941,  ..., -0.3412, -0.2314, -0.1686],
          ...,
          [ 0.0980,  0.1059,  0.0902,  ..., -0.0980, -0.2471, -0.3804],
          [ 0.1059,  0.1216,  0.1137,  ..., -0.2314, -0.3647, -0.4510],
          [ 0.0824,  0.0980,  0.1294,  ..., -0.3490, -0.4039, -0.4353]],

         [[-0.4039, -0.3020, -0.0431,  ..., -0.6392, -0.5765, -0.4745],
          [-0.0980, -0.0902, -0.0039,  ..., -0.6235, -0.4431, -0.4196],
          [-0.1059, -0.2157, -0.2392,  ..., -0.6471, -0.5686, -0.5059],
          ...,
          [-0.3804, -0.37

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Model Output tensor([[-0.1392, -0.0370, -0.1433]], grad_fn=<AddmmBackward0>)
